In [1]:
import msgpack
import open3d as o3d
# import trimesh
from IPython.display import display, Math
import numpy as np
import matplotlib.pyplot as plt
import time
import threading
import asyncio

plt.style.use('seaborn-whitegrid')
%config InlineBackend.figure_format = 'retina'

# %autoawait off

import socket
BUFFER_LEN = 512 #in bytes

# # !pip install nest-asyncio
# import nest_asyncio
# nest_asyncio.apply()

In [2]:
ip_local = "127.0.0.1"
port_local = 32000

ip_remote = "127.0.0.1"
port_remote = 32001

local_address = (ip_local,port_local)
remote_address = (ip_remote,port_remote)

# https://adaickalavan.github.io/programming/udp-socket-programming-in-cpp-and-python/
# Python UDP Receiver
def initUDP(IP, port):
    #Create a datagram socket
    sock = socket.socket(socket.AF_INET, # Internet
                         socket.SOCK_DGRAM) # UDP
#     #Enable immediate reuse of IP address
    sock.setsockopt(socket.SOL_SOCKET,socket.SO_REUSEADDR,1)
    #Bind the socket to the port
    sock.bind((IP, port))
    #Set a timeout so the socket does not block indefinitely when trying to receive data
    sock.settimeout(1)

    return sock

def readUDP(sock):
    data = sock.recv(BUFFER_LEN)
    return data

In [12]:
# sock = initUDP(ip_local, port_local) # socket that receives udp packet
s = time.perf_counter()
#Create a datagram socket
sock = socket.socket(socket.AF_INET, # Internet
                     socket.SOCK_DGRAM) # UDP

sock.setsockopt(socket.SOL_SOCKET,socket.SO_REUSEADDR,0)
sock.settimeout(1)
sock.bind(local_address)
# sock.connect(remote_address)
try:
    data = sock.recv(BUFFER_LEN)
    data_unpacked = msgpack.unpackb(data)
    print(data_unpacked)
except Exception as e:
#     print(e)
    pass
# sock.shutdown(socket.SHUT_RDWR) # shutdown the connection
# sock.close()# close the connection


elapsed = time.perf_counter() - s

print(f"#--- executed in {elapsed*1000:0.2f} ms ---#")


[14, 0.0, [-0.7166480909868898, -0.33594497217166014, -0.33594497217166014, -0.33594497217166014], [0.0, 0.0, 0.0, 0.0], [-0.697434953013431, 0.94188161446786, 0.94188161446786, 0.94188161446786, 0.0, 0.0], [1.027548286604283, -0.35667430705870695, -0.35667430705870695], [-0.33594497217166014, -0.33594497217166014, -0.33594497217166014]]
#--- executed in 8.97 ms ---#


In [12]:
# loop = asyncio.get_event_loop()
# loop.create_task(test())

In [4]:
RuntimeError

RuntimeError

In [3]:
import asyncio
data_unpacked = None
class EchoServerProtocol:
    
    def connection_made(self, transport):
        self.transport = transport

    def datagram_received(self, data, addr):
        global data_unpacked
        global flag_received
        flag_received = True
        data_unpacked = msgpack.unpackb(data)


loop = asyncio.get_event_loop()
print("Starting UDP server")

# One protocol instance will be created to serve all client requests
listen = loop.create_datagram_endpoint(
    EchoServerProtocol, local_addr=('127.0.0.1', 32000))



try:
    transport, protocol = loop.run_until_complete(listen)
except Exception as e:
    print(e)
    
try:
    loop.run_forever()
except KeyboardInterrupt:
    transport.close()
    loop.close()

Starting UDP server
This event loop is already running


RuntimeError: This event loop is already running

In [12]:
data_unpacked

[14,
 0.0,
 [0.4746756997623797,
  0.4746756997623797,
  0.6027931319996285,
  -0.9763766097103586],
 [0.0, 0.0, 0.0, 0.0],
 [0.8801607694365247,
  0.8801607694365247,
  -0.7978975122245202,
  -0.2160757182343867,
  0.0,
  0.0],
 [0.5393056771506246, 0.5393056771506246, -0.7554768911599372],
 [0.4746756997623797, 0.4746756997623797, -0.9763766097103586]]

### test receiving udp packet

In [6]:
try:
#     data_receive = readUDP(sock)
    data_receive= sock.recv(BUFFER_LEN)
#     print(data_receive)
    data_receive_unpacked = msgpack.unpackb(data_receive)
    print(data_receive_unpacked)
except socket.timeout:
    print("rec_socket timed out!")
# sock.shutdown(socket.SHUT_RDWR) # shutdown the connection
# sock.close()# close the connection

NameError: name 'sock' is not defined

### test sending udp packet

In [5]:
sen_sock = initUDP(ip_remote, port_local) # socket that send udp packet

data_send = [12,time.time(),[0,0,3,2]]
print(data_send)
# data_send = [0.5,0.0,1.0,2.0]
# data_send = [1,2,3,4]

# data_send = 20

packer = msgpack.Packer(use_single_float=False,use_bin_type =True)
data_send_packed = packer.pack(data_send) # https://msgpack-python.readthedocs.io/en/latest/api.html#msgpack.Packer
# data_send_packed = msgpack.packb(data_send)

sen_sock.sendto(data_send_packed,remote_address)

sen_sock.shutdown(socket.SHUT_RDWR) # shutdown the connection
sen_sock.close()# close the connection

[12, 1596439783.598737, [0, 0, 3, 2]]


In [6]:
data_send_packed

b'\x93\x0c\xcbA\xd7\xc9\xef9\xe6Q\xb5\x94\x00\x00\x03\x02'

In [4]:
import ipywidgets.widgets as widgets
import ipywidgets
from IPython.display import display

In [5]:
w_max = 300.0/60.0*np.pi*2

flag_start = True
flag_reset = False

# stop button
button_stop = widgets.Button(
    description='stop',
    icon='stop',
    button_style='warning')

def OnClickButtonStop(b):
    global flag_start
    flag_start=False
button_stop.on_click(OnClickButtonStop)  

# reset button
button_reset = widgets.Button(
    description='reset',
    icon='refresh',
    button_style='info')

def OnClickButtonReset(b):
    global flag_reset
    global w0_widget
    global w1_widget
    global w2_widget
    global w3_widget
    
    flag_start=True
    w0_widget.value=0
    w1_widget.value=0
    w2_widget.value=0
    w3_widget.value=0

    
    
button_reset.on_click(OnClickButtonReset) 

w0_widget = widgets.FloatSlider(min=-w_max, max=w_max, step=.01, value=0,
                                   continuous_update=True, description="front left")
w1_widget = widgets.FloatSlider(min=-w_max, max=w_max, step=.01, value=0,
                                   continuous_update=True, description="back left")
w2_widget = widgets.FloatSlider(min=-w_max, max=w_max, step=.01, value=0,
                                   continuous_update=True, description="front right")
w3_widget = widgets.FloatSlider(min=-w_max, max=w_max, step=.01, value=0,
                                   continuous_update=True, description="back right")
caption_list = [
    widgets.Label(value='[Joint Angle]'),
    widgets.Label(value='[Joint speed]'),
    widgets.Label(value='[Orientation]'),
    widgets.Label(value='[Accelerration]'),
    widgets.Label(value='[Position]'),
]
                

ui = widgets.VBox([w0_widget, w1_widget, w2_widget, w3_widget]+caption_list)

packer = msgpack.Packer(use_single_float=False,use_bin_type =True)


sen_sock = initUDP(ip_remote, port_local) # socket that send udp packet


def f(w0, w1, w2, w3):    
    t_now = time.time()
    data_send = (2,t_now,(w0, w1, w2, w3))
#     data_send_packed = msgpack.packb(data_send)
    data_send_packed = packer.pack(data_send) # https://msgpack-python.readthedocs.io/en/latest/api.html#msgpack.Packer
#     display(data_send_packed)
    
    sen_sock.sendto(data_send_packed,remote_address)
    
#     display((f"{time.time():3.3f}", (w0, w1, w2, w3)))
    
#     sen_sock.shutdown(socket.SHUT_RDWR) # shutdown the connection
#     sen_sock.close()# close the connection
    
out = widgets.interactive_output(f, {
    'w0': w0_widget,
    'w1': w1_widget,
    'w2': w2_widget,
    'w3': w3_widget
})
# out.layout.height = '350px'
# out.layout.width = '350px'

buttons_ui = widgets.HBox((button_stop,button_reset))

app = widgets.VBox(
    children = (ui, out,buttons_ui))

# display(app)

In [6]:
display(app)

In [20]:
update_interval = 1e-3 - 1e-5
def AppControl():
    time_previous = time.time()
    global flag_start
    flag_start=True
    
    rec_sock = initUDP(ip_remote, port_local) # socket that receives udp packet
    while flag_start:
        time_now = time.time()
        if time_now-time_previous>update_interval:
            print(f"{time_now-time_previous}\r",end='')
            time_previous = time_now
            try:

                data_receive = readUDP(rec_sock)
            #     print(data_receive)
                data_receive_unpacked = msgpack.unpackb(data_receive)
                # data checking the header should be a certain number(TODO)
                if data_receive_unpacked[0]:
                    #caption_list.value=f"{data_receive_unpacked}"
                    for k,caption in enumerate(caption_list):
                        caption.value = " ".join(f"{x:<+10.3f}" for x in data_receive_unpacked[2+k])
            except socket.timeout:
                print("rec_socket timed out!\r",end='')
    rec_sock.shutdown(socket.SHUT_RDWR) # shutdown the connection
    rec_sock.close()# close the connection

In [22]:
control_thread = threading.Thread(target=AppControl)
display(app)
control_thread.start()

In [43]:

button = widgets.Button(description="Click Me!")
output = widgets.Output()

display(button, output)

def on_button_clicked(b):
    with output:
        print("started")

button.on_click(on_button_clicked)

Button(description='Click Me!', style=ButtonStyle())

Output()

Button clicked.
Button clicked.
Button clicked.
Button clicked.


In [ ]:
started = True

In [47]:

# # start button
# button_start = widgets.Button(
#     description='start',
#     icon='start',
#     button_style='warning')
# button_start



Button(button_style='warning', description='stop', icon='stop', style=ButtonStyle())

In [31]:
tmp = msgpack.packb((0.00,-128.00,0.00,0.00))

In [32]:
msgpack.unpackb(tmp)

[0.0, -128.0, 0.0, 0.0]

In [35]:
packer.pack((0.00,-128.00,0.00,0.00))

b'\x94\xca\x00\x00\x00\x00\xca\xc3\x00\x00\x00\xca\x00\x00\x00\x00\xca\x00\x00\x00\x00'

In [40]:
items = [widgets.Label(str(i)) for i in range(4)]
widgets.Box(items)

Box(children=(Label(value='0'), Label(value='1'), Label(value='2'), Label(value='3')))

In [42]:
items = [widgets.Label(str(i)) for i in range(8)]
widgets.GridBox(items, layout=widgets.Layout(grid_template_columns="repeat(3, 100px)"))

GridBox(children=(Label(value='0'), Label(value='1'), Label(value='2'), Label(value='3'), Label(value='4'), La…

In [45]:
items[0].value='dfs'

In [49]:
caption_list = widgets.Label(value='The values of slider1 and slider2 are synchronized')
sliders1, slider2 = widgets.IntSlider(description='Slider 1'),\
                    widgets.IntSlider(description='Slider 2')
l = ipywidgets.link((sliders1, 'value'), (slider2, 'value'))
display(caption_list, sliders1, slider2)

Label(value='The values of slider1 and slider2 are synchronized')

IntSlider(value=0, description='Slider 1')

IntSlider(value=0, description='Slider 2')

In [51]:

a = widgets.IntSlider(description="Delayed", continuous_update=False)
b = widgets.IntText(description="Delayed", continuous_update=False)
c = widgets.IntSlider(description="Continuous", continuous_update=True)
d = widgets.IntText(description="Continuous", continuous_update=True)

ipywidgets.link((a, 'value'), (b, 'value'))
ipywidgets.link((a, 'value'), (c, 'value'))
ipywidgets.link((a, 'value'), (d, 'value'))
widgets.VBox([a,b,c,d])

## rotation

In [111]:
from scipy.spatial.transform import Rotation as R
def RotateFromVec(vec, angle):
    vec = np.asarray(vec, dtype=np.float64)
    vec_norm = np.linalg.norm(vec)
    if vec_norm != 1:
        vec = vec/vec_norm
    vec *= angle
    h = np.eye(4)
    h[:3, :3] = R.from_rotvec(vec).as_dcm()
    return h       

# https://www.euclideanspace.com/maths/geometry/rotations/conversions/matrixToQuaternion/
def RotMatrixToQuarternion(m):
    """
    m is a 3x3 rotation matrix or 4x4 homogenious transformation matrix
    """
    w = np.sqrt(1+m[0,0]+m[1,1]+m[2,2])/2.0
    w4 = 4*w
    x = (m[2,1]-m[1,2])/w4
    y = (m[0,2]-m[2,0])/w4
    z = (m[1,0]-m[0,1])/w4
    return np.array([w,x,y,z])

# https://www.euclideanspace.com/maths/geometry/rotations/conversions/quaternionToAngle/index.htm
def QuarternionToAxisAngle(q):
    if(q[0]>1):
        q = q/np.linalg.norm(q)
    angle = 2.0 * np.arccos(q[0])
    s = np.sqrt(1-q[0]**2)
    if(s<1e-8):
        return q[1:],angle
    else:
        return q[1:]/s,angle

In [5]:
m = RotateFromVec([0,1,1],np.pi/2.)
q = RotMatrixToQuarternion(m)
QuarternionToAxisAngle(q)

C:\Users\boxi\.conda\envs\conda_env\lib\site-packages\ipykernel_launcher.py:9: DeprecationWarning: `as_dcm` is deprecated!
as_dcm is renamed to as_matrix in scipy 1.4.0 and will be removed in scipy 1.6.0
  if __name__ == '__main__':


(array([0.        , 0.70710678, 0.70710678]), 1.5707963267948966)

In [15]:
import asyncio

async def test():
    print('hello')
    await asyncio.sleep(1)
    print('world')

loop = asyncio.get_event_loop()
loop.create_task(test())

<Task pending coro=<test() running at <ipython-input-15-2e9d42bb6be1>:3>>

hello
world
